In [1]:
from utils import *
import pandas as pd
import json
import folium
from folium import Choropleth, Circle, Marker,CircleMarker
from folium.plugins import HeatMap, MarkerCluster
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

In [2]:
# 버스 노선 데이터

# 피클 데이터 불러오기
bus_passengers_20230605_final=call_df("bus_passengers_20230605_final")
# 데이터프레임 중복제거한 노선ID 추출
arr=bus_passengers_20230605_final["노선_ID"].tolist()

result1=[]
for id in arr:
    if id not in result1:
        result1.append(id)
print(len(result1))

# 데이터프레임 동일한 그룹끼리 분류하기
final=bus_passengers_20230605_final.groupby("노선_ID")
group_name=[]
for id in result1:
    globals()['final_{}'.format(id)]=final.get_group(id)
    group_name.append('final_{}'.format(id))

494


In [3]:
# 폴리움 관련 데이터 불러오기
tb_subway_month_avg=pd.read_csv("data/tb_subway_month_avg")
total_bus_month_avg=pd.read_csv("data/total_bus_month_avg")
total_bus_month_avg_low200=pd.read_csv("data/total_bus_month_avg_low200")
total_bus_month_avg_top200=pd.read_csv("data/total_bus_month_avg_top200")


## 버스 노선 정하기
좌표를 바탕으로 노선을 구하고 그 노선내에 좌표범위를 설정하여 가장 가까운 노드를 정하는 것

In [4]:
# 8~9시 버스 승하차
# from config import vworld_key
vworld_key = "BD606474-76A5-3146-848E-21906146E125"
# Draw a basemap
m = folium.Map(location=[37.58, 127.0], tiles='cartodbpositron', zoom_start=11)

# 배경지도 타일 설정하기
layer = "midnight"
tileType = "png"
tiles = f"http://api.vworld.kr/req/wmts/1.0.0/{vworld_key}/{layer}/{{z}}/{{y}}/{{x}}.{tileType}"
attr = "Vworld"

folium.TileLayer(
    tiles=tiles,
    attr=attr,
    overlay=True,
    control=True
).add_to(m)

# 버스 승차인원 상위 200
for q in total_bus_month_avg_top200.values.tolist():
    CircleMarker(location=(q[22],q[21]),
           radius=q[4]/500,tooltip=q[0],popup=[q[22],q[21]],color='green',fill_color='green',opacity=2).add_to(m)

# 버스 하차인원 상위 200
for l in total_bus_month_avg_low200.values.tolist():
    CircleMarker(location=(l[22],l[21]),
           radius=l[13]/500,tooltip=l[0],popup=[l[22],l[21]],color='crimson',fill_color='crimson',opacity=1.5).add_to(m)
    
    # 버스 노선 그리기
for k in group_name:
    folium.PolyLine(locations=eval(k)[["Y좌표","X좌표"]].values.tolist(),tooltip=set([i for i in eval(k)["노선_ID"]]),opacity=0.09, color='yellow').add_to(m)

m
# m.save("bus_8.html")

In [12]:
end_y=37.54787148999999
end_x=126.8353186
start_y = 37.46931873
start_x = 126.9686577

In [13]:
total_bus_month_avg_low200[total_bus_month_avg_low200["Y좌표"]==end_y]

,NODE_ID,USE_MON,SIX_RIDE_NUM,SEVEN_RIDE_NUM,EIGHT_RIDE_NUM,NINE_RIDE_NUM,전체 탑승자수,ride_rate_06_07,ride_rate_07_08,ride_rate_08_09,...,EIGHT_ALIGHT_NUM,NINE_ALIGHT_NUM,하차인원 수,alight_rate_06_07,alight_rate_07_08,alight_rate_08_09,alight_rate_09_10,ARS_ID,X좌표,Y좌표
21,115900195,202181.758621,102.517241,314.482759,593.0,364.413793,6964.0,0.014738,0.045053,0.085113,...,4370.172414,2622.655172,29064.827586,0.052088,0.135849,0.150181,0.09024,16943.0,126.835319,37.547871


In [7]:
total_bus_month_avg_top200[total_bus_month_avg_top200["Y좌표"]==start_y]

,NODE_ID,USE_MON,SIX_RIDE_NUM,SEVEN_RIDE_NUM,EIGHT_RIDE_NUM,NINE_RIDE_NUM,전체 탑승자수,ride_rate_06_07,ride_rate_07_08,ride_rate_08_09,...,EIGHT_ALIGHT_NUM,NINE_ALIGHT_NUM,하차인원 수,alight_rate_06_07,alight_rate_07_08,alight_rate_08_09,alight_rate_09_10,ARS_ID,X좌표,Y좌표
94,120900079,202181.758621,660.172414,1161.37931,1330.551724,972.413793,9815.344828,0.067344,0.117643,0.134938,...,34.689655,57.931034,1656.758621,0.015701,0.017189,0.020951,0.034423,21506.0,126.968658,37.469319


In [8]:
if start_x>=end_x and start_y>= end_y:
    total_bus_month_avg_top200[total_bus_month_avg_top200[total_bus_month_avg_top200["Y좌표"]==start_y]["NODE_ID"]]

In [9]:
total_bus_month_avg_top200.loc[total_bus_month_avg_top200["NODE_ID"]==total_bus_month_avg_top200[total_bus_month_avg_top200["Y좌표"]==start_y]["NODE_ID"].values.tolist()[0],:]

,NODE_ID,USE_MON,SIX_RIDE_NUM,SEVEN_RIDE_NUM,EIGHT_RIDE_NUM,NINE_RIDE_NUM,전체 탑승자수,ride_rate_06_07,ride_rate_07_08,ride_rate_08_09,...,EIGHT_ALIGHT_NUM,NINE_ALIGHT_NUM,하차인원 수,alight_rate_06_07,alight_rate_07_08,alight_rate_08_09,alight_rate_09_10,ARS_ID,X좌표,Y좌표
94,120900079,202181.758621,660.172414,1161.37931,1330.551724,972.413793,9815.344828,0.067344,0.117643,0.134938,...,34.689655,57.931034,1656.758621,0.015701,0.017189,0.020951,0.034423,21506.0,126.968658,37.469319


In [10]:
total_bus_month_avg_top200[total_bus_month_avg_top200["Y좌표"]==start_y]["NODE_ID"].values.tolist()[0]

120900079

In [15]:
total_bus_month_avg_low200.loc[total_bus_month_avg_low200["NODE_ID"]==total_bus_month_avg_low200[total_bus_month_avg_low200["Y좌표"]==end_y]["NODE_ID"].values.tolist()[0],:]

,NODE_ID,USE_MON,SIX_RIDE_NUM,SEVEN_RIDE_NUM,EIGHT_RIDE_NUM,NINE_RIDE_NUM,전체 탑승자수,ride_rate_06_07,ride_rate_07_08,ride_rate_08_09,...,EIGHT_ALIGHT_NUM,NINE_ALIGHT_NUM,하차인원 수,alight_rate_06_07,alight_rate_07_08,alight_rate_08_09,alight_rate_09_10,ARS_ID,X좌표,Y좌표
21,115900195,202181.758621,102.517241,314.482759,593.0,364.413793,6964.0,0.014738,0.045053,0.085113,...,4370.172414,2622.655172,29064.827586,0.052088,0.135849,0.150181,0.09024,16943.0,126.835319,37.547871
